In [6]:
"""
Импортируем необходимые библиотеки для создания генеративно-состязательной сети
Код разработан в основном с использованием библиотеки PyTorch
"""
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms
#from model import discriminator, generator
import numpy as np
import matplotlib.pyplot as plt

In [7]:
"""
Определяем, доступны ли какие-либо графические процессоры
"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [36]:
"""
Сетевые архитектуры
Ниже приведены архитектуры дискриминатора и генератора

class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 1)
        self.activation = nn.LeakyReLU(0.1)
    def forward(self, x):
        x = x.view(-1, 784)
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return nn.Sigmoid()(x)
class generator(nn.Module):
    def __init__(self):
        super(generator, self).__init__()
        self.fc1 = nn.Linear(128, 1024)
        self.fc2 = nn.Linear(1024, 2048)
        self.fc3 = nn.Linear(2048, 784)
        self.activation = nn.ReLU()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        x = x.view(-1, 1, 28, 28)
        return nn.Tanh()(x)"""

'\nСетевые архитектуры\nНиже приведены архитектуры дискриминатора и генератора\n\nclass discriminator(nn.Module):\n    def __init__(self):\n        super(discriminator, self).__init__()\n        self.fc1 = nn.Linear(784, 512)\n        self.fc2 = nn.Linear(512, 1)\n        self.activation = nn.LeakyReLU(0.1)\n    def forward(self, x):\n        x = x.view(-1, 784)\n        x = self.activation(self.fc1(x))\n        x = self.fc2(x)\n        return nn.Sigmoid()(x)\nclass generator(nn.Module):\n    def __init__(self):\n        super(generator, self).__init__()\n        self.fc1 = nn.Linear(128, 1024)\n        self.fc2 = nn.Linear(1024, 2048)\n        self.fc3 = nn.Linear(2048, 784)\n        self.activation = nn.ReLU()\n    def forward(self, x):\n        x = self.activation(self.fc1(x))\n        x = self.activation(self.fc2(x))\n        x = self.fc3(x)\n        x = x.view(-1, 1, 28, 28)\n        return nn.Tanh()(x)'

In [37]:
z_dim = 100

class generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 784),
            nn.Tanh()
        )
        
    def forward(self, z):
        x = self.fc(z)
        return x

class discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = x.view(-1, 784)
        out = self.fc(x)
        return out

In [38]:
"""
Determine if any GPUs are available
"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


"""
Hyperparameter settings
"""
epochs = 150
lr = 2e-4
batch_size = 64
loss = nn.BCELoss()

# Model
G = generator().to(device)
D = discriminator().to(device)

G_optimizer = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
D_optimizer = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))


"""
Image transformation and dataloader creation
Note that we are training generation and not classification, and hence
only the train_loader is loaded
"""
# Transform
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Load data
train_set = datasets.MNIST('mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)


"""
Network training procedure
Every step both the loss for disciminator and generator is updated
Discriminator aims to classify reals and fakes
Generator aims to generate images as realistic as possible
"""
"""for epoch in range(epochs):
    for idx, (imgs, _) in enumerate(train_loader):
        idx += 1

        # Training the discriminator
        # Real inputs are actual images of the MNIST dataset
        # Fake inputs are from the generator
        # Real inputs should be classified as 1 and fake as 0
        real_inputs = imgs.to(device)
        real_outputs = D(real_inputs)
        real_label = torch.ones(real_inputs.shape[0], 1).to(device)

        noise = (torch.rand(real_inputs.shape[0], 128) - 0.5) / 0.5
        noise = noise.to(device)
        fake_inputs = G(noise)
        fake_outputs = D(fake_inputs)
        fake_label = torch.zeros(fake_inputs.shape[0], 1).to(device)

        outputs = torch.cat((real_outputs, fake_outputs), 0)
        targets = torch.cat((real_label, fake_label), 0)

        D_loss = loss(outputs, targets)
        D_optimizer.zero_grad()
        D_loss.backward()
        D_optimizer.step()

        # Training the generator
        # For generator, goal is to make the discriminator believe everything is 1
        noise = (torch.rand(real_inputs.shape[0], 128)-0.5)/0.5
        noise = noise.to(device)

        fake_inputs = G(noise)
        fake_outputs = D(fake_inputs)
        fake_targets = torch.ones([fake_inputs.shape[0], 1]).to(device)
        G_loss = loss(fake_outputs, fake_targets)
        G_optimizer.zero_grad()
        G_loss.backward()
        G_optimizer.step()

        if idx % 100 == 0 or idx == len(train_loader):
            print('Epoch {} Iteration {}: discriminator_loss {:.3f} generator_loss {:.3f}'.format(epoch, idx, D_loss.item(), G_loss.item()))

    if (epoch+1) % 10 == 0:
        torch.save(G, 'Generator_epoch_{}.pth'.format(epoch))
        print('Model saved.')"""


NameError: name 'Generator' is not defined

In [39]:
"""
Процедура обучения сети.
Каждый шаг потери обновляется как для дискиминатора, так и для генератора.
Дискриминатор стремится классифицировать реальные и fakes
Генератор стремится генерировать как можно более реалистичные изображения
"""
epochs = 100
for epoch in range(epochs):
    for idx, (imgs, _) in enumerate(train_loader):
        idx += 1
        # Обучаем дискриминатор
        # real_inputs - изображения из набора данных MNIST 
        # fake_inputs - изображения от генератора
        # real_inputs должны быть классифицированы как 1, а fake_inputs - как 0
        real_inputs = imgs.to(device)
        real_outputs = D(real_inputs)
        real_label = torch.ones(real_inputs.shape[0], 1).to(device)
        noise = (torch.rand(real_inputs.shape[0], 128) - 0.5) / 0.5
        noise = noise.to(device)
        fake_inputs = G(noise)
        fake_outputs = D(fake_inputs)
        fake_label = torch.zeros(fake_inputs.shape[0], 1).to(device)
        outputs = torch.cat((real_outputs, fake_outputs), 0)
        targets = torch.cat((real_label, fake_label), 0)
        D_loss = loss(outputs, targets)
        D_optimizer.zero_grad()
        D_loss.backward()
        D_optimizer.step()
        # Обучаем генератор
        # Цель генератора получить от дискриминатора 1 по всем изображениям
        noise = (torch.rand(real_inputs.shape[0], 128)-0.5)/0.5
        noise = noise.to(device)
        fake_inputs = G(noise)
        fake_outputs = D(fake_inputs)
        fake_targets = torch.ones([fake_inputs.shape[0], 1]).to(device)
        G_loss = loss(fake_outputs, fake_targets)
        G_optimizer.zero_grad()
        G_loss.backward()
        G_optimizer.step()
        if idx % 100 == 0 or idx == len(train_loader):
            print('Epoch {} Iteration {}: discriminator_loss {:.3f} generator_loss {:.3f}'.format(epoch, idx, D_loss.item(), G_loss.item()))
    if (epoch+1) % 10 == 0:
        torch.save(G, 'Generator_epoch_{}.pth'.format(epoch))
        print('Model saved.')

Epoch 0 Iteration 100: discriminator_loss 0.701 generator_loss 0.869
Epoch 0 Iteration 200: discriminator_loss 0.655 generator_loss 0.844
Epoch 0 Iteration 300: discriminator_loss 0.625 generator_loss 0.833
Epoch 0 Iteration 400: discriminator_loss 0.676 generator_loss 0.752
Epoch 0 Iteration 500: discriminator_loss 0.664 generator_loss 0.638
Epoch 0 Iteration 600: discriminator_loss 0.698 generator_loss 0.717
Epoch 0 Iteration 700: discriminator_loss 0.653 generator_loss 0.727
Epoch 0 Iteration 800: discriminator_loss 0.660 generator_loss 0.726
Epoch 0 Iteration 900: discriminator_loss 0.675 generator_loss 0.849
Epoch 0 Iteration 938: discriminator_loss 0.652 generator_loss 0.724
Epoch 1 Iteration 100: discriminator_loss 0.696 generator_loss 0.729
Epoch 1 Iteration 200: discriminator_loss 0.676 generator_loss 0.806
Epoch 1 Iteration 300: discriminator_loss 0.674 generator_loss 0.697
Epoch 1 Iteration 400: discriminator_loss 0.672 generator_loss 0.716
Epoch 1 Iteration 500: discriminat

PicklingError: Can't pickle <class '__main__.generator'>: it's not the same object as __main__.generator

In [35]:
x = G.forward(noise).detach().numpy()
x.reshape()

ValueError: cannot reshape array of size 25088 into shape (1,28,28)

In [34]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

ax = sns.heatmap(x, annot=True, fmt="d")
plt.title("Визуализация массива",fontsize=12)
plt.savefig("visual_numpy_array.png", bbox_inches='tight', dpi=100)
plt.show()

ValueError: Must pass 2-d input. shape=(32, 1, 28, 28)